In [1]:
%load_ext autoreload 
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as dists
import pytorch_lightning as pl
from src.api import *
from torchvision.utils import make_grid
from experiments.vanilla_vae import VAE

In [2]:
x = torch.zeros(16, 1, 28, 28)
z_dim = 100
e = Chain([
    nn.Conv2d(1, 16, kernel_size=3, padding=1),
    nn.LeakyReLU(),
    nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
    nn.LeakyReLU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
    nn.LeakyReLU(),
    lambda x: torch.flatten(x, start_dim=1),
    nn.Linear(7*7*64, 2*z_dim),
    lambda x: torch.chunk(x, chunks=2, dim=1),
    lambda mean, logstd: dists.Normal(mean, torch.exp(logstd))
])

d = Chain([
    nn.Linear(z_dim, 7*7*64),
    nn.LeakyReLU(),
    lambda x: torch.reshape(x, (-1, 64, 7, 7)),
    nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
    nn.LeakyReLU(),
    nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1),
    nn.LeakyReLU(),
    nn.Conv2d(16, 1, kernel_size=3, padding=1),
    nn.Sigmoid()
])

pz = dists.Normal(
    torch.zeros(z_dim),
    torch.ones(z_dim)
)
    

In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
transform = transforms.ToTensor()
data_train = datasets.MNIST(root="datasets/", train=True, transform=transform)
data_test = datasets.MNIST(root="datasets/", train=False, transform=transform)
train_loader = DataLoader(data_train, batch_size=64, shuffle=True, num_workers=8)

model = VAE(e, d, pz)
trainer = pl.Trainer(gpus=1)
trainer.fit(model, train_loader)

/home/xbt/anaconda3/envs/neuro/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type  | Params
----------------------------------
0 | encoder | Chain | 650 K 
1 | decoder | Chain | 357 K 
----------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Tot

Training: -1it [00:00, ?it/s]

/home/xbt/anaconda3/envs/neuro/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1047: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
